En esta libreta vamos a experimentar con el análisis de tópicos a partir de un corpus en español muy interesante para realizar análisis de tópicos: Contiene transcripciones de las informes diarios sobre coronavirus en México desde el inicio de febrero del 2020 organizados por archivos csv, sin embargo para este trabajo vamos a considerar únicamente las participaciones realizadas por el Dr. Hugo López-Gatell Ramírez.

El corpus se encuentra [en este proyecto de github](https://github.com/NOSTRODATA/informe_diario_sobre_coronavirus_en_mexico) y pueden encontrar más información a cerca de [NOSTRODATA](https://www.nostrodata.com) en su página de internet.


## 1. Obtención del *corpus*

Para obtener el *corpus* es necesario clonar el proyecto original, se usa el comando:

```
git clone https://github.com/NOSTRODATA/informe_diario_sobre_coronavirus_en_mexico.git
```

Una vez clonado, es necesario extraer la información de los archivos `csv`. La generación del *corpus* sin normalizar ya la dejo desarollada.

En mi caso realicé una separación de los archivos csv para contar solamente con las participaciones que relaizo el Dr. Hugo López-Gatell Ramirez a inicios de la pandemia en el 2020 y las últimas del persente año 2021.

In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('./2020/*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

['./2020\\HUGO LOPEZ-GATELL RAMIREZ1.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ10.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ11.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ12.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ13.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ14.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ15.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ16.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ17.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ18.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ19.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ2.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ20.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ21.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ22.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ3.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ4.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ5.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ6.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ7.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ8.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ9.csv']


In [3]:
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de 
# archivo a través de globbing y 
# el resultado final será la lista dataframes
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola
list_data
 
pd.concat(list_data,ignore_index=True)['Texto']

0                     Muchas gracias, doctor Reyes Terán.
1       Ahora le pido al doctor José Luis Alomía si no...
2                                 Gracias, doctor Alomía.
3       Abrimos para preguntas. Por favor, compañera, ...
4        Le contesto la segunda y le paso a su compatr...
                              ...                        
2049    No hay una previsión anticipada de cierre de a...
2050    Y aquí ratifico la instrucción del señor presi...
2051                                             Gracias.
2052                                                  ---
2053                                              VE 823.
Name: Texto, Length: 2054, dtype: object

In [4]:
informes20 = pd.concat(list_data,ignore_index=True)['Texto']

In [5]:
informe20 = np.array(informes20)
informe20

array([' Muchas gracias, doctor Reyes Terán.',
       'Ahora le pido al doctor José Luis Alomía si nos resume la situación actual en México.',
       ' Gracias, doctor Alomía.', ..., 'Gracias.', '---', 'VE 823.'],
      dtype=object)

In [6]:
import random

informe20[random.randint(0, 2053)]

'Que no se propague a partir de un caso a sus contactos y que, si esos contactos llegaron a infectarse, que ellos no propaguen todavía más.'

## 2. Procesaiento de texto

Ahora es necesario normalizar el texto con fin de utilizarlo en modelado de tópicos.

El texto lo vamos a tratar con al menos los siguientes requisitos:

1. Utilizar todas las palabras en minúsculas
2. Eliminar los signos de puntuación
3. Eliminar palabras vacias 
    
Realiza esta normalización utilizando el módulo *spacy* con el modelo `es_core_news_sm`, pero sientete con la libertad de cargar el modelo 
`es_core_news_md` o inclusive el modelo `es_core_news_lg`. 

El corpus procesado (*normalizado* también se le dice) se guardará en una lista de textos que llamaremos `informe20_tratado`

In [7]:
import spacy
nlp = spacy.load("es_core_news_sm")

In [8]:
def procesa_informe(doc): 
    
    return [
        token.norm_ for token in doc
        if token.is_alpha and not token.like_num and not token.is_stop and
           token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']
    ]

informe20_tratado = [procesa_informe(doc) for doc in nlp.pipe(informes20)]

Y utiliza esta celda para probar si se normalizaron vien las participaciones realizadas.

In [9]:
i = random.randint(0, (len(informe20_tratado) - 1))
print(i)
print(informes20[i])
print("********************")
print(informe20_tratado[i])

1728
Como hemos dicho también en otras ocasiones, es importante que la ciudadanía, que el pueblo tenga la información a su alcance, esta es una información técnica, científica, que utilizamos quienes somos especialistas en epidemiología, pero es importante que la población sepa directamente cuál es la lógica, por qué estamos haciendo las cosas.
********************
['ocasiones', 'importante', 'ciudadanía', 'pueblo', 'información', 'alcance', 'información', 'técnica', 'científica', 'utilizamos', 'especialistas', 'epidemiología', 'importante', 'población', 'sepa', 'lógica']


Ya realizado el tratamiento de las participaciones nos quedaremos con aquellas que cuenten con más de 3 palabras normalizadas.

In [10]:
np_lists = np.array(informe20_tratado)

filt = []
for i in range(0,len(informe20_tratado)):
    filt_i = len(np_lists[i]) >= 4
    filt.append(filt_i)
informe20_tratado = list(np_lists[filt])
# print("\n", informe20_tratado)

<ipython-input-10-de42bce1a1d8>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_lists = np.array(informe20_tratado)


## 3. Modelado de tópicos con LDA

Ahora, ya con el corpus tratado, desarrolla tu modelo LDA en *gensim*, siguiendo los siguientes pasos:

1. Crea un diccionario con el informe tratado, donde elimines a todas las
   palabras que no aparezcan en al menos `min_df` documentos, y las palabras que 
   aparezcan el más del `max_df` porciento de documentos (por default `min_df = 5`
   y `max_df = 0.3`).
2. Genera un corpus listo para su uso, aplicando el metodo de bolsa de palabras.
3. Aplica el método para modelar con LDA. Establece el número de iteraciones para
   el método de estimación así como el numero de pasos que realiza el algoritmo
   de optimización iterativa (por default 100 y 5, respectivamente).
   
Guarda el modelo en la variable `modelo_lda` (opcionalmente puedes guardar el modelo si lo quieres usar más adelante)


In [11]:
import warnings
warnings.filterwarnings('ignore')

from gensim import models, corpora

# Genera el diccionario de palabras
diccionario20 = corpora.Dictionary(informe20_tratado)
diccionario20.filter_extremes(no_below=5, no_above=0.3)
 
# Extrae las características en forma de BOW
corpus20 = [diccionario20.doc2bow(doc) for doc in informe20_tratado]

Y ahora hacemos el modelado de tópicos con LDA

In [12]:
# Número de tópicos
n_topicos = 15

# Genera el modelo LDA
modelo_lda_20 = models.LdaModel(corpus=corpus20, num_topics=n_topicos, id2word=diccionario20, iterations=100, passes=10)

# Si quieres guardar el modelo se hace esto
#modelo_lda.save("sonetosLDA.model")

Y podemos usar otros métodos como el de *Hierachical Dirichlet Process*

In [13]:
modelo_hdp_20 = models.hdpmodel.HdpModel(corpus20, diccionario20, T=n_topicos)

Y ahora veamos los modelos como se definen por sus primeras 6 palabras clave 

In [14]:
for (i, topico) in modelo_lda_20.print_topics(num_topics=n_topicos, num_words=6):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.119*"casos" + 0.021*"epidemia" + 0.019*"contactos" + 0.016*"cantidad" + 0.016*"medidas" + 0.015*"número"
----------topico 1--------------------
0.035*"consejo" + 0.025*"caso" + 0.023*"salud" + 0.020*"persona" + 0.017*"reunión" + 0.015*"salubridad"
----------topico 2--------------------
0.022*"salud" + 0.016*"blanca" + 0.016*"evento" + 0.013*"seguridad" + 0.013*"nacional" + 0.013*"medidas"
----------topico 3--------------------
0.024*"personas" + 0.022*"síntomas" + 0.020*"epidemia" + 0.020*"persona" + 0.015*"méxico" + 0.014*"dolor"
----------topico 4--------------------
0.026*"prueba" + 0.024*"diagnóstico" + 0.024*"laboratorios" + 0.020*"salud" + 0.013*"indre" + 0.013*"doctor"
----------topico 5--------------------
0.039*"favor" + 0.022*"medidas" + 0.022*"compañera" + 0.021*"ahorita" + 0.020*"compañero" + 0.019*"colega"
----------topico 6--------------------
0.054*"doctor" + 0.032*"gracias" + 0.026*"alomía" + 0.025*"social" + 0.016*"población" + 

In [15]:
for (i, topico) in modelo_hdp_20.print_topics(num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.006*presente + 0.006*cumbre + 0.006*acción + 0.005*control + 0.005*tomamos
----------topico 1--------------------
0.006*interno + 0.005*masiva + 0.005*oficinas + 0.005*conocimiento + 0.005*responder
----------topico 2--------------------
0.005*reconocimiento + 0.005*norma + 0.005*rastrear + 0.005*cuidado + 0.005*masivos
----------topico 3--------------------
0.007*esperamos + 0.006*servicios + 0.005*plan + 0.005*identificación + 0.005*promoción
----------topico 4--------------------
0.006*historia + 0.006*marina + 0.005*adicional + 0.005*organización + 0.005*inició
----------topico 5--------------------
0.006*imss + 0.005*explicado + 0.005*graves + 0.005*valadez + 0.005*específica
----------topico 6--------------------
0.007*fila + 0.006*ocurre + 0.006*años + 0.005*casas + 0.005*valadez
----------topico 7--------------------
0.008*específica + 0.006*noticia + 0.006*doctora + 0.005*masiva + 0.005*administración
----------topico 8-----------------

Y ahora veamos como se clasifica un documento utilizando tanto LDA como HDP

In [16]:
i = random.randint(0, (len(informe20) - 1))

respuesta20 = informes20[i]
ids_20 = diccionario20.doc2bow(informe20_tratado[i])

print("Informe: \n\n" + respuesta20)

topicos_lda_20 = modelo_lda_20[ids_20]
topicos_lda_20.sort(key=lambda x:x[1], reverse=True)

topicos_hdp_20 = modelo_hdp_20[ids_20]
topicos_hdp_20.sort(key=lambda x:x[1], reverse=True)

print()
print("Pertenece a los tópicos (con el modelo LDA)")
for (topico, peso) in topicos_lda_20:
    print("\tTopico {} con un peso de {}".format(topico, peso))

print("Pertenece a los tópicos (con el modelo HDP)")
for (topico, peso) in topicos_hdp_20:
    print("\tTopico {} con un peso de {}".format(topico, peso))

Informe: 

Porque queríamos estar en el aislamiento, no queríamos que esta persona que había sido confirmada ahorita esté transitando y además nos facilita el estudio de contactos, porque con él podemos trabajar. Ha habido excelente cooperación, quiero destacarlo, estas tres personas han cooperado perfectamente, muy conscientes del problema de salud pública y estamos haciendo eso.

Pertenece a los tópicos (con el modelo LDA)
	Topico 2 con un peso de 0.6569092869758606
	Topico 9 con un peso de 0.19863973557949066
	Topico 3 con un peso de 0.011111648753285408
	Topico 13 con un peso de 0.011111633852124214
	Topico 6 con un peso de 0.011111619882285595
	Topico 14 con un peso de 0.011111617088317871
	Topico 0 con un peso de 0.011111611500382423
	Topico 1 con un peso de 0.011111611500382423
	Topico 11 con un peso de 0.0111116087064147
	Topico 4 con un peso de 0.011111607775092125
	Topico 12 con un peso de 0.011111607775092125
	Topico 8 con un peso de 0.011111601255834103
	Topico 10 con un pe

Tambien se puede obtener la lista de palabras que describen a cada tópico.

Mejor aún, utiliza *pyLDAvis* para visualizar y analizar los tópicos desarrollados.

In [17]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(modelo_lda_20, corpus20, diccionario20)
pyLDAvis.display(vis_data)

In [18]:
vis_data = gensimvis.prepare(modelo_hdp_20, corpus20, diccionario20)
pyLDAvis.display(vis_data)

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 4 Evaluando modelos

Para evaluar un modelo, la forma más sencilla es utilizando el modelo para evaluar la perplejidad del corpus

In [19]:
print(f"El logaritomo de la perplejidad del corpus al modelo LDA es {modelo_lda_20.log_perplexity(corpus20)}")

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


El logaritomo de la perplejidad del corpus al modelo LDA es -7.01847546072328


Otro método es analizando la [coherencia de los tópicos](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf), ponemos algunos resultados:

In [20]:
cm_lda_20 = models.CoherenceModel(model=modelo_lda_20, texts=informe20_tratado, coherence='c_v')
cm_hdp_20 = models.CoherenceModel(model=modelo_hdp_20, texts=informe20_tratado, coherence='c_v')


print("Coherencia del modelo LDA (C_V): {}".format(cm_lda_20.get_coherence()))
for (topico, coherencia) in enumerate(cm_lda_20.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

print("\n\nCoherencia del modelo HDP (C_V): {}".format(cm_hdp_20.get_coherence()))
for (topico, coherencia) in enumerate(cm_hdp_20.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherencia del modelo LDA (C_V): 0.36032404096578474
	Tópico 0, coherencia 0.319929865533091
	Tópico 1, coherencia 0.3998385745219176
	Tópico 2, coherencia 0.5030356429514476
	Tópico 3, coherencia 0.3071558105452746
	Tópico 4, coherencia 0.4034314815442143
	Tópico 5, coherencia 0.409079254954505
	Tópico 6, coherencia 0.3669291620819259
	Tópico 7, coherencia 0.33195403811331864
	Tópico 8, coherencia 0.31859296890717276
	Tópico 9, coherencia 0.2568799111246775
	Tópico 10, coherencia 0.4408786333428038
	Tópico 11, coherencia 0.29922017940702844
	Tópico 12, coherencia 0.43714734982421233
	Tópico 13, coherencia 0.35133534720329396
	Tópico 14, coherencia 0.2594523944318865


Coherencia del modelo HDP (C_V): 0.7394722269631987
	Tópico 0, coherencia 0.7633879574355289
	Tópico 1, coherencia 0.7269353072559783
	Tópico 2, coherencia 0.7992046557601548
	Tópico 3, coherencia 0.7422311220074278
	Tópico 4, coherencia 0.7308519731814207
	Tópico 5, coherencia 0.7158231039102573
	Tópico 6, coherencia 0.

Con la coherencia se puede obtener el conjunto de palabras que mejor definen cada tópico

In [21]:
print("Las palabra que definen los tópicos en LDA\n")
t_palabras = cm_lda_20.top_topics_as_word_lists(model=modelo_lda_20, dictionary=diccionario20)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")
    
print("\n\nLas palabra que definen los tópicos en HDP\n")
t_palabras = cm_hdp_20.top_topics_as_word_lists(model=modelo_hdp_20, dictionary=diccionario20)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")

Las palabra que definen los tópicos en LDA

0 - casos, epidemia, contactos, cantidad, medidas, número, sospechosos, transmisión, países, aislamiento, méxico, vigilancia, semanas, distancia, situación, contención, estudio, mitigación, curva, fase
1 - consejo, caso, salud, persona, reunión, salubridad, casa, tuvimos, méxico, país, secretario, sanitaria, autoridad, nacional, sesión, guía, epidemia, pregunta, cubrebocas, presidente
2 - salud, blanca, evento, seguridad, nacional, medidas, ángeles, comité, técnicos, actividad, país, semana, respuesta, importante, viajó, trabajando, valadez, tomar, plan, manos
3 - personas, síntomas, epidemia, persona, méxico, dolor, ocurre, casa, atención, gobierno, fiebre, información, enfermedad, situación, médica, quédate, coronavirus, covid, casos, sector
4 - prueba, diagnóstico, laboratorios, salud, indre, doctor, privados, laboratorio, china, países, reyes, personas, pruebas, pública, terán, vámonos, referencia, temperatura, italia, actividades
5 - fav

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Realizamos los mismos procedimientos para el análisis de tópicos del 2021

In [22]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('./2021/*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['./2021\\HUGO LOPEZ-GATELL RAMIREZ1.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ10.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ11.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ12.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ13.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ14.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ15.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ16.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ17.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ18.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ19.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ2.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ20.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ21.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ22.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ3.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ4.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ5.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ6.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ7.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ8.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ9.csv']


In [23]:
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y 
# el resultado final será la lista dataframes
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola
list_data
 
pd.concat(list_data,ignore_index=True)['Texto']

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                    Se escucha muy bien, Juan. Adelante.
1       Excelente. Muchas gracias, gracias a todas, bu...
2       Antes de hacerles un par de preguntas que quis...
3       Por eso me parece muy acertado, como lo formul...
4       Y dado que somos una república federal, tambié...
                              ...                        
2320    Y sobre lo que me preguntaba, ya me pasaron la...
2321    Le invitamos a usted, si tiene 50 años de edad...
2322    Nuestro interés, el interés del Gobierno de Mé...
2323    Muchas gracias. Nos vemos mañana. Gracias, doc...
2324                                                  ---
Name: Texto, Length: 2325, dtype: object

In [24]:
informes21 = pd.concat(list_data,ignore_index=True)['Texto']

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
informe21 = np.array(informes21)
informe21

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Se escucha muy bien, Juan. Adelante.',
       'Excelente. Muchas gracias, gracias a todas, bueno a la doctora\xa0Giorguli\xa0y a todos ustedes.',
       'Antes de hacerles un par de preguntas que quisiéramos hacerles a nuestros ponentes, una por mi parte, otra por parte del doctor López Ridaura, retomo sobre lo que ya fue explicado para destacar lo que para nosotros es una convicción: la importancia de escuchar a distintas personalidades en la sociedad, en el mundo académico y científico, por ejemplo, representado por estas 14 instituciones, tanto nacionales, como incluso internacionales, tales como la Comisión Económica para América Latina de Naciones Unidas o el Banco Mundial, y también escuchar a la sociedad, la sociedad civil organizada nos ha aportado también elementos valiosísimos, en algunos casos de asesoría, de análisis, de reflexión, de crítica y también en muchos otros casos de soluciones prácticas, se ha sumado ampliamente.',
       ...,
       'Nuestro interés, el 

In [26]:
informe21[random.randint(0, (len(informe21) - 1))]

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'Cuando pase la emergencia,\xa0posiblemente aquellas vacunas que demostraron ser las más exitosas solicitarán o no registro permanentes y sólo en ese momento ya se pueden tener para la venta al público.'

In [27]:
def procesa_informe(doc): 
    
    return [
        token.norm_ for token in doc
        if token.is_alpha and not token.like_num and not token.is_stop and
           token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']
    ]

informe21_tratado = [procesa_informe(doc) for doc in nlp.pipe(informes21)]

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
np_lists = np.array(informe21_tratado)

filt = []
for i in range(0,len(informe21_tratado)):
    filt_i = len(np_lists[i]) >= 6
    filt.append(filt_i)
informe21_tratado = list(np_lists[filt])
# print("\n", informe21_tratado)

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
i = random.randint(0, (len(informe21_tratado) - 1))
print(i)
print(informes21[i])
print("********************")
print(informe21_tratado[i])

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


678
Y concluimos comentando que el programa de vacunación contra COVID en México, que es público y gratuito, va avanzando a buen ritmo tal como lo habíamos planeado desde el 8 de diciembre del año pasado. Vamos logrando una cobertura ya superior a 11 por ciento de la población de México y seguiremos avanzando cada vez más rápido, porque cada vez están llegando las vacunas contra COVID más rápidamente.
********************
['lunes', 'comentadas', 'recibidas', 'liberadas', 'cansino', 'recibidas', 'sputnik', 'inicio', 'contemplada', 'pfizer', 'acabamos', 'recibir', 'comunicación', 'pasarán', 'jueves', 'aclaro', 'diapositiva', 'actualizaremos', 'acabamos', 'recibir', 'comunicación', 'pfizer', 'pasan', 'jueves', 'miércoles', 'jueves']


In [30]:
import warnings
warnings.filterwarnings('ignore')

from gensim import models, corpora

# Genera el diccionario de palabras
diccionario21 = corpora.Dictionary(informe21_tratado)
diccionario21.filter_extremes(no_below=5, no_above=0.3)
 
# Extrae las características en forma de BOW
corpus21 = [diccionario21.doc2bow(doc) for doc in informe21_tratado]

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# Número de tópicos
n_topicos = 15

# Genera el modelo LDA
modelo_lda_21 = models.LdaModel(corpus=corpus21, num_topics=n_topicos, id2word=diccionario21, iterations=100, passes=10)

# Si quieres guardar el modelo se hace esto
#modelo_lda.save("sonetosLDA.model")

In [32]:
modelo_hdp_21 = models.hdpmodel.HdpModel(corpus21, diccionario21, T=n_topicos)

In [33]:
for (i, topico) in modelo_lda_21.print_topics(num_topics=n_topicos, num_words=6):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.021*"gracias" + 0.019*"semáforo" + 0.018*"personal" + 0.017*"entidades" + 0.015*"verde" + 0.013*"epidemia"
----------topico 1--------------------
0.032*"méxico" + 0.020*"pfizer" + 0.018*"vacuna" + 0.017*"pública" + 0.017*"salud" + 0.016*"gobierno"
----------topico 2--------------------
0.068*"personas" + 0.049*"vacunación" + 0.046*"personal" + 0.026*"educativo" + 0.025*"salud" + 0.020*"entidades"
----------topico 3--------------------
0.024*"vacunación" + 0.020*"comunidades" + 0.020*"salud" + 0.016*"sarampión" + 0.016*"pregunta" + 0.015*"vacunas"
----------topico 4--------------------
0.108*"dosis" + 0.062*"millones" + 0.055*"vacuna" + 0.047*"vacunas" + 0.028*"semana" + 0.022*"personas"
----------topico 5--------------------
0.049*"personas" + 0.033*"vacunación" + 0.014*"vacunar" + 0.014*"doctor" + 0.012*"eventos" + 0.012*"vacuna"
----------topico 6--------------------
0.027*"preguntas" + 0.021*"juan" + 0.019*"abrir" + 0.017*"hernández" + 0.016*

In [34]:
for (i, topico) in modelo_hdp_21.print_topics(num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.005*ciudades + 0.005*número + 0.005*epidémicas + 0.005*legales + 0.005*informar
----------topico 1--------------------
0.006*hablando + 0.005*fila + 0.004*basta + 0.004*persona + 0.004*unión
----------topico 2--------------------
0.006*fda + 0.005*completado + 0.005*medios + 0.004*técnico + 0.004*recibimos
----------topico 3--------------------
0.005*existía + 0.004*tipo + 0.004*características + 0.004*agradecemos + 0.004*terminando
----------topico 4--------------------
0.006*semejante + 0.005*derecho + 0.005*resto + 0.005*empezar + 0.005*porcentaje
----------topico 5--------------------
0.006*federal + 0.005*atención + 0.005*país + 0.004*hospitalización + 0.004*junio
----------topico 6--------------------
0.007*priorización + 0.006*local + 0.005*pasa + 0.004*tamaño + 0.004*duración
----------topico 7--------------------
0.005*indicaciones + 0.005*llaman + 0.005*vemos + 0.004*logremos + 0.004*sitios
----------topico 8--------------------
0.007*

In [35]:
i = random.randint(0, (len(informe21) - 1))

respuesta21 = informes21[i]
ids_21 = diccionario21.doc2bow(informe21_tratado[i])

print("Informe: \n\n" + respuesta21)

topicos_lda_21 = modelo_lda_21[ids_21]
topicos_lda_21.sort(key=lambda x:x[1], reverse=True)

topicos_hdp_21 = modelo_hdp_21[ids_21]
topicos_hdp_21.sort(key=lambda x:x[1], reverse=True)

print()
print("Pertenece a los tópicos (con el modelo LDA)")
for (topico, peso) in topicos_lda_21:
    print("\tTopico {} con un peso de {}".format(topico, peso))

print("Pertenece a los tópicos (con el modelo HDP)")
for (topico, peso) in topicos_hdp_21:
    print("\tTopico {} con un peso de {}".format(topico, peso))

Informe: 

Por cierto, bueno, ya que menciona Argentina, puedo dar un preanuncio. La próxima semana, a partir del lunes, vamos a tener la visita de la ministra de Salud de Argentina, la doctora Carla Vizzotti, gran amiga y compañera. Hemos coincidido en muchos eventos de salud pública a lo largo del tiempo y fue quien tuvimos el privilegio que nos recibiera en Argentina, en Buenos Aires, en la primera semana de enero, cuando fuimos a la misión especial que comentó el presidente López Obrador para ir a conseguir la vacuna Sputnik V.

Pertenece a los tópicos (con el modelo LDA)
	Topico 7 con un peso de 0.6724763512611389
	Topico 5 con un peso de 0.28812459111213684
Pertenece a los tópicos (con el modelo HDP)
	Topico 1 con un peso de 0.700864939362032
	Topico 10 con un peso de 0.26004932494289484


In [36]:
vis_data = gensimvis.prepare(modelo_lda_21, corpus21, diccionario21)
pyLDAvis.display(vis_data)

In [37]:
vis_data = gensimvis.prepare(modelo_hdp_21, corpus21, diccionario21)
pyLDAvis.display(vis_data)

In [38]:
print(f"El logaritomo de la perplejidad del corpus al modelo LDA es {modelo_lda_21.log_perplexity(corpus21)}")

El logaritomo de la perplejidad del corpus al modelo LDA es -6.947430475244754


In [39]:
cm_lda_21 = models.CoherenceModel(model=modelo_lda_21, texts=informe21_tratado, coherence='c_v')
cm_hdp_21 = models.CoherenceModel(model=modelo_hdp_21, texts=informe21_tratado, coherence='c_v')


print("Coherencia del modelo LDA (C_V): {}".format(cm_lda_21.get_coherence()))
for (topico, coherencia) in enumerate(cm_lda_21.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

print("\n\nCoherencia del modelo HDP (C_V): {}".format(cm_hdp_21.get_coherence()))
for (topico, coherencia) in enumerate(cm_hdp_21.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

Coherencia del modelo LDA (C_V): 0.3164649209595442
	Tópico 0, coherencia 0.2914092216568901
	Tópico 1, coherencia 0.2685661472994768
	Tópico 2, coherencia 0.48615999414375494
	Tópico 3, coherencia 0.23507936440805977
	Tópico 4, coherencia 0.5593877454625231
	Tópico 5, coherencia 0.26249503049821815
	Tópico 6, coherencia 0.36483910003958436
	Tópico 7, coherencia 0.2347864639931092
	Tópico 8, coherencia 0.3715067807251822
	Tópico 9, coherencia 0.2274803046409059
	Tópico 10, coherencia 0.27641036669608526
	Tópico 11, coherencia 0.31733716160207875
	Tópico 12, coherencia 0.3325746685057618
	Tópico 13, coherencia 0.275456897660442
	Tópico 14, coherencia 0.2434845670610911


Coherencia del modelo HDP (C_V): 0.7470531702992517
	Tópico 0, coherencia 0.7125520901949063
	Tópico 1, coherencia 0.6702846335693202
	Tópico 2, coherencia 0.7196261273517208
	Tópico 3, coherencia 0.7483488653956087
	Tópico 4, coherencia 0.7297755029654225
	Tópico 5, coherencia 0.7349001782311516
	Tópico 6, coherencia 0

In [40]:
print("Las palabra que definen los tópicos en LDA\n")
t_palabras = cm_lda_21.top_topics_as_word_lists(model=modelo_lda_21, dictionary=diccionario21)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")
    
print("\n\nLas palabra que definen los tópicos en HDP\n")
t_palabras = cm_hdp_21.top_topics_as_word_lists(model=modelo_hdp_21, dictionary=diccionario21)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")

Las palabra que definen los tópicos en LDA

0 - gracias, semáforo, personal, entidades, verde, epidemia, federativas, potosí, san, luis, entidad, amarillo, educativo, campeche, importante, personas, favor, salud, federativa, vacunas
1 - méxico, pfizer, vacuna, pública, salud, gobierno, vacunas, proceso, cofepris, unidos, tuvimos, población, cansino, conferencia, interés, planta, múltiples, velocidad, europea, liberación
2 - personas, vacunación, personal, educativo, salud, entidades, línea, federativas, semana, covid, sistema, vacunando, propósito, población, vacunar, adultas, sector, mayores, público, nacional
3 - vacunación, comunidades, salud, sarampión, pregunta, vacunas, baja, condiciones, actividad, méxico, niños, gracias, importante, pública, servicios, epidémica, transmisión, términos, niñas, seguras
4 - dosis, millones, vacuna, vacunas, semana, personas, esquema, cansino, semanas, completo, sputnik, número, recibimos, embarque, calendario, sinovac, cantidad, recibido, v, pfize